In [ ]:
import pickle
import random

import numpy as np
import sklearn.preprocessing
import matplotlib.pyplot as plt

import stengel.model.pitch_data
import stengel.model.pitch_outcome

In [ ]:
with open("../data/python/pitch_data_2009.p", "rb") as pitch_file:
    pitch_data = stengel.model.pitch_data.PitchData.from_dict(pickle.load(pitch_file))
with open("../data/python/compressed_renders.p", "rb") as compressed_density_file:
    compressed_density = pickle.load(compressed_density_file)

In [ ]:
# Prep the main pitch data
pitch_data.filter_nulls(in_place=True)
pitch_counts = pitch_data.pitches_per_pitcher()
high_count_pitchers = [i for i, pc in enumerate(pitch_counts) if pc > 3000]
pitch_data.filter_by_pitcher_id(high_count_pitchers, in_place=True)

In [ ]:
# Prep the density array
density_compressed_array = np.array([compressed_density[name].reshape([-1])
                                     for name in pitch_data.pitchers])
density_compressed_array = density_compressed_array - np.mean(density_compressed_array)
pitch_data.pitch_density = density_compressed_array

In [ ]:
# Get indices for train, validation, and test sets
train_obs = int(0.6 * pitch_data.num_observations)
valid_obs = int(0.2 * pitch_data.num_observations)
train_indices = np.array(range(train_obs))
valid_indices = np.array(range(train_obs, train_obs + valid_obs))
test_indices = np.array(range(train_obs + valid_obs, pitch_data.num_observations))

# Split the data into train, validation and test randomly
pitch_data.shuffle(31415)
train_data = pitch_data.filter_rows(train_indices, reassign_ids=False)
valid_data = pitch_data.filter_rows(valid_indices, reassign_ids=False)
test_data = pitch_data.filter_rows(test_indices, reassign_ids=False)
valid_data.shuffle_each_epoch = False
test_data.shuffle_each_epoch = False

# Normalize the data according to the training data distributions
scaler = sklearn.preprocessing.StandardScaler()
scaler.fit(train_data.pitch_data)
train_data.pitch_data = scaler.transform(train_data.pitch_data)
valid_data.pitch_data = scaler.transform(valid_data.pitch_data)
test_data.pitch_data = scaler.transform(test_data.pitch_data)

num_batters = len(train_data.batters)
num_pitchers = len(train_data.pitchers)

In [ ]:
# Logistic regression model
logistic_model = stengel.model.pitch_outcome.PitchOutcomeModel(
        batch_size=64, learning_rate=0.1, hidden_nodes=[]
)
logistic_model.train(train_data, valid_data, 100000, 5000)
logistic_model.save("../model_fits/logistic.tf")

In [ ]:
# Single hidden layer model
single_hidden_model = stengel.model.pitch_outcome.PitchOutcomeModel(
        batch_size=64, learning_rate=0.1, hidden_nodes=[192]
)
single_hidden_model.train(train_data, valid_data, 100000, 5000)
single_hidden_model.save("../model_fits/single_hidden.tf")

In [ ]:
# Pitcher and batter embedding model
embedding_model = stengel.model.pitch_outcome.PitchOutcomeModel(
        batch_size=64, learning_rate=0.1, hidden_nodes=[192],
        num_batters=num_batters, batter_embed_size=24,
        num_pitchers=num_pitchers, pitcher_embed_size=24
)
embedding_model.train(train_data, valid_data, 220000, 5000)
embedding_model.save("../model_fits/embedding.tf")

In [ ]:
# Pitch density model, no hidden layer
density_direct_model = stengel.model.pitch_outcome.PitchOutcomeModel(
        batch_size=64, learning_rate=0.1, hidden_nodes=[192],
        num_batters=num_batters, batter_embed_size=24,
        num_pitchers=num_pitchers, pitcher_embed_size=24,
        density_size=[322]
)
density_direct_model.train(train_data, valid_data, 220000, 5000)
density_direct_model.save("../model_fits/density_direct.tf")

In [ ]:
# Pitch density model
density_model = stengel.model.pitch_outcome.PitchOutcomeModel(
        batch_size=64, learning_rate=0.1, hidden_nodes=[192],
        num_batters=num_batters, batter_embed_size=24,
        num_pitchers=num_pitchers, pitcher_embed_size=24,
        density_size=[322], density_hidden_nodes=[64]
)
density_model.train(train_data, valid_data, 220000, 5000)
density_model.save("../model_fits/density.tf")

In [ ]:
print("Logistic model:        {:0.4f}".format(logistic_model.score(test_data)))
print("Single hidden model:   {:0.4f}".format(single_hidden_model.score(test_data)))
print("Embedding model:       {:0.4f}".format(embedding_model.score(test_data)))
print("Direct density model:  {:0.4f}".format(density_direct_model.score(test_data)))
print("Hidden density model:  {:0.4f}".format(density_model.score(test_data)))

In [ ]:
plt.figure(figsize=(6, 4))
plt.plot(logistic_model.fit_log["batch_number"], 
         logistic_model.fit_log["validation_score"], 
         label="Logistic")
plt.plot(single_hidden_model.fit_log["batch_number"], 
         single_hidden_model.fit_log["validation_score"],
         label="Single Hidden Layer")
plt.plot(embedding_model.fit_log["batch_number"], 
         embedding_model.fit_log["validation_score"],
         label="Embedding")
plt.plot(density_direct_model.fit_log["batch_number"], 
         density_direct_model.fit_log["validation_score"],
         label="Density")
plt.plot(density_model.fit_log["batch_number"], 
         density_model.fit_log["validation_score"],
         label="Density w/ Hidden")
plt.legend()
plt.ylabel("Perplexity")
plt.xlabel("Training Batches")
plt.title("Validation Performance of Final Models Over Training")
plt.show()

In [ ]:
plt.figure(figsize=(6, 4))
plt.plot(logistic_model.fit_log["batch_number"], 
         logistic_model.fit_log["validation_score"], 
         label="Logistic")
plt.plot(single_hidden_model.fit_log["batch_number"], 
         single_hidden_model.fit_log["validation_score"],
         label="Single Hidden Layer")
plt.plot(embedding_model.fit_log["batch_number"], 
         embedding_model.fit_log["validation_score"],
         label="Embedding")
plt.plot(density_direct_model.fit_log["batch_number"], 
         density_direct_model.fit_log["validation_score"],
         label="Density")
plt.plot(density_model.fit_log["batch_number"], 
         density_model.fit_log["validation_score"],
         label="Density w/ Hidden")
plt.legend()
plt.ylabel("Perplexity")
plt.xlabel("Training Batches")
plt.title("Validation Performance of Final Models Over Training")
plt.ylim([2.75, 3.0])
plt.show()

In [ ]:
logistic_predictions = logistic_model.predict(test_data)
single_hidden_predictions = single_hidden_model.predict(test_data)
embedding_predictions = embedding_model.predict(test_data)
density_direct_predictions = density_direct_model.predict(test_data)
density_predictions = density_model.predict(test_data)

In [ ]:
test_outcomes = test_data.pitch_outcomes[:32768]
logistic_scores = [logistic_predictions[i, o] for i, o in enumerate(test_outcomes)]
single_hidden_scores = [single_hidden_predictions[i, o] for i, o in enumerate(test_outcomes)]
embedding_scores = [embedding_predictions[i, o] for i, o in enumerate(test_outcomes)]
density_direct_scores = [density_direct_predictions[i, o] for i, o in enumerate(test_outcomes)]
density_scores = [density_predictions[i, o] for i, o in enumerate(test_outcomes)]
combined_scores = np.array([logistic_scores, single_hidden_scores, embedding_scores,
                            density_direct_scores, density_scores]).transpose()

In [ ]:
# Bootstrap perplexity scores
bootstrapped_scores = []
for _ in range(500):
    bootstrap_indices = np.random.choice(np.array(32768), 32768, replace=True)
    bootstrap_scores = combined_scores[bootstrap_indices, :]
    scores_by_model = np.exp(-np.mean(np.log(bootstrap_scores), axis=0))
    bootstrapped_scores.append(scores_by_model)
bootstrapped_scores = np.array(bootstrapped_scores)

In [ ]:
plt.boxplot(bootstrapped_scores,
           labels=["Logistic", "Single Hidden", "Embedding", "Density", "Density w/ Hidden"])
plt.ylabel("Perplexity")
plt.xlabel("Model")
plt.title("Bootstrapped Testing Performance of Final Models")
plt.show()

In [ ]:
plt.boxplot(bootstrapped_scores[:, 1:],
           labels=["Single Hidden", "Embedding", "Density", "Density w/ Hidden"])
plt.ylabel("Perplexity")
plt.xlabel("Model")
plt.title("Bootstrapped Testing Performance of Final Models")
plt.show()

In [ ]:
ball_and_strike_indices = test_outcomes <= 1
balls_and_strikes_actual = test_outcomes[ball_and_strike_indices]
balls_and_strikes_embedded = np.argmax(embedding_predictions[ball_and_strike_indices, :2], axis=1)
np.mean(balls_and_strikes_embedded == balls_and_strikes_actual)

In [ ]:
pitch_locations = test_data.pitch_data[:32768, 31:33]
ball_and_strike_locations = pitch_locations[ball_and_strike_indices, ]

In [ ]:
actual_balls = balls_and_strikes_actual == 0
actual_strikes = balls_and_strikes_actual == 1
plt.scatter(ball_and_strike_locations[actual_balls, 1],
            ball_and_strike_locations[actual_balls, 0],
            alpha=0.05,
            color='b')
plt.scatter(ball_and_strike_locations[actual_strikes, 1],
            ball_and_strike_locations[actual_strikes, 0],
            alpha=0.05,
            color='r')
plt.ylim(-3, 3)
plt.xlim(-3, 3)
plt.title("Actual Ball and Strike Calls")
plt.show()

In [ ]:
embedded_balls = balls_and_strikes_embedded == 0
embedded_strikes = balls_and_strikes_embedded == 1
plt.scatter(ball_and_strike_locations[embedded_balls, 1],
            ball_and_strike_locations[embedded_balls, 0],
            alpha=0.05,
            color='b')
plt.scatter(ball_and_strike_locations[embedded_strikes, 1],
            ball_and_strike_locations[embedded_strikes, 0],
            alpha=0.05,
            color='r')
plt.ylim(-3, 3)
plt.xlim(-3, 3)
plt.title("Predicted Balls and Strikes")
plt.show()

In [ ]:
right_calls = balls_and_strikes_actual == balls_and_strikes_embedded
wrong_calls = balls_and_strikes_actual != balls_and_strikes_embedded
plt.scatter(ball_and_strike_locations[right_calls, 1],
            ball_and_strike_locations[right_calls, 0],
            alpha=0.05,
            color='b')
plt.scatter(ball_and_strike_locations[wrong_calls, 1],
            ball_and_strike_locations[wrong_calls, 0],
            alpha=0.05,
            color='r')
plt.ylim(-3, 3)
plt.xlim(-3, 3)
plt.title("Incorrect Pitch Call Predictions")
plt.show()